# A. Reading in the Data

In [1]:
import pandas as pd
import numpy

claims = pd.read_csv("tsa_claims.csv")
claims.head()

C:\Users\hp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Claim Amount,Status,Close Amount,Disposition
0,0909802M,4-Jan-02,12/12/2002 0:00,EWR,Newark International Airport,Continental Airlines,Property Damage,Checkpoint,Other,$350.00,Approved,$350.00,Approve in Full
1,0202417M,2-Feb-02,1/16/2004 0:00,SEA,Seattle-Tacoma International,NaN,Property Damage,Checked Baggage,Luggage (all types including footlockers),$100.00,Settled,$50.00,Settle
2,0202445M,4-Feb-02,11/26/2003 0:00,STL,Lambert St. Louis International,American Airlines,Property Damage,Checked Baggage,Cell Phones,$278.88,Settled,$227.92,Settle
3,0909816M,7-Feb-02,1/6/2003 0:00,MIA,Miami International Airport,American Airlines,Property Damage,Checkpoint,Luggage (all types including footlockers),$50.00,Approved,$50.00,Approve in Full
4,2005032379513,18-Feb-02,2/5/2005 0:00,MCO,Orlando International Airport,Delta (Song),Property Damage,Checkpoint,Baby - Strollers; car seats; playpen; etc.,$84.79,Approved,$84.79,Approve in Full


In [2]:
# Checking for Null values

claims.isnull().sum()

Claim Number         0
Date Received      263
Incident Date     2183
Airport Code      8524
Airport Name      8524
Airline Name     34374
Claim Type        7913
Claim Site         740
Item              3966
Claim Amount      4043
Status               5
Close Amount     68952
Disposition      72908
dtype: int64

# B. Cleaning the Data

In [3]:
# First task undertaken is to determine what percentage of amount of claims are either approved or
# settled by the TSA.

# Since the values in the "Claim Amount" and "Close Amount" columns is of string datatype with
# dollar signs and other punctuations, we first need to reduce the string values purely to digits,
# from where they can be converted to numeric values.

for c in ["Claim Amount", "Close Amount"]:
    claims[c] = claims[c].str.replace("$", "")
    claims[c] = claims[c].str.replace(" ", "")
    claims[c] = claims[c].str.replace(";", "")
    claims[c] = claims[c].str.replace("'", "")
    
    # Removing rows with null values.
     
    claims.dropna(subset=[c], inplace=True)
    
    # Converting values to numeric.
    
    claims[c] = pd.to_numeric(claims[c], errors="coerce")
    
claims.isnull().sum()

Claim Number         0
Date Received      110
Incident Date      324
Airport Code      4014
Airport Name      4014
Airline Name     28336
Claim Type        5468
Claim Site         283
Item              3233
Claim Amount         0
Status               0
Close Amount         0
Disposition       4848
dtype: int64

# 3. Calculating claims approval percentages

In [4]:
A = claims["Claim Amount"][claims["Disposition"]=="Approved in Full"].sum()
B = claims["Claim Amount"][claims["Disposition"]=="Settle"].sum()
C = claims["Close Amount"][claims["Disposition"]=="Approved in Full"].sum()
D = claims["Close Amount"][claims["Disposition"]=="Settle"].sum()

x = A + B
y = C + D

print((y/x)*100)

30.812183545968992


__Thus, of the total amount claimed, only 30.81% was reimbursed.__

In [5]:
# The second task is to determine the number of claims accepted.

E = claims["Close Amount"][claims["Disposition"]=="Approved in Full"].count()
F = claims["Close Amount"][claims["Disposition"]=="Settle"].count()

x1 = claims["Claim Amount"].count()
y1 = E + F

print((y1/x1)*100)

20.707127239621812


__Thus, of the total claims, only 20.7% were accepted or settled.__

# 4. Five airports with highest claims

In [6]:
# The third task is to find which airport had the highest filed claims and settled/approved
# claims, thus determining the relationship between an airport's size and approved claims.

# Cleaning the row by removing null values.
for airport in ["Airport Code", "Airport Name"]:
    claims.dropna(subset=[airport], inplace=True)
    
claims.isnull().sum()

Claim Number         0
Date Received      102
Incident Date      224
Airport Code         0
Airport Name         0
Airline Name     25880
Claim Type        4834
Claim Site         261
Item              3162
Claim Amount         0
Status               0
Close Amount         0
Disposition       4596
dtype: int64

In [7]:
claims["Airport Code"].value_counts().head()

LAX    6416
JFK    4956
ORD    4815
EWR    4684
MIA    4104
Name: Airport Code, dtype: int64

__Thus, the five airports with highest number of claims were
Los Angeles Intl,
Kennedy Intl,
O'Hare Intl,
Newark Intl and Mimai.__

In [8]:
claims["Airport Code"][claims["Disposition"] == "Approve in Full"].value_counts().head(5)

LAX    1380
ORD    1262
SEA    1126
EWR    1048
PHX    1012
Name: Airport Code, dtype: int64

__Thus, the five airports with highest number of aprroved claims were Los Angeles Intl, O'Hare Intl, Tacoma Intl, Newark Intl and Pheonix Intl.__